In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import glob
import os
import os
import cv2
import glob
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mlxtend.plotting import plot_confusion_matrix
from scipy import ndimage
import skimage
from skimage.io import imread
from skimage.transform import resize
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit as s_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop, Adam
from keras.layers import Conv2D , SeparableConv2D, BatchNormalization, UpSampling2D, MaxPool2D
from keras.layers import MaxPooling2D, Average, Input, Concatenate, LeakyReLU, Add, ELU, PReLU, ReLU
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras import backend as BE
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, MaxPooling2D
from keras.models import Model
from keras.datasets import cifar10

c:\users\chandrachurh\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\chandrachurh\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\chandrachurh\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [4]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [5]:
X_train /= 255
X_test /= 255

In [6]:
BE.clear_session()

In [7]:
def residual_block(mod_, f_in, f_out, strides_ = (1,1), use_shortcut_ = False):   
    shortcut_ = mod_
    
    mod_ = SeparableConv2D(f_in, (1,1), strides=(1,1), padding = "same")(mod_)
    mod_ = BatchNormalization()(mod_)
    mod_ = LeakyReLU()(mod_)
    if use_shortcut_ == True or strides_ != (1,1):
        shortcut_ = SeparableConv2D(f_out, (1,1), strides=strides_, padding = "same")(shortcut_)
        shortcut_ = BatchNormalization()(shortcut_)
        
    mod_ = Add()([shortcut_, mod_])
    mod_ = ReLU()(mod_)
    
    return mod_

In [8]:
def model_build():
    
    #od_ = SeparableConv2D(16, (2,2), strides = (2,2), padding = "same")(in_)
    #od_ = BatchNormalization()(mod_)
    #od_ = ReLU()(mod_)
    
    #od_ = residual_block(mod_, 16, 32, use_shortcut_ = True)
    #od_ = MaxPooling2D()(mod_)
    #od_ = residual_block(mod_, 32, 64, use_shortcut_ = True)
    #od_ = MaxPooling2D()(mod_)
    #od_ = residual_block(mod_, 64, 96, use_shortcut_ = True)
    #od_ = MaxPooling2D()(mod_)
    #od_ = residual_block(mod_, 96, 128, use_shortcut_ = True)
    
    #od_ = GlobalAveragePooling2D()(mod_)
    #od_ = Dense(512, activation = "relu")(mod_)
    #od_ = Dropout(0.5)(mod_)
    #red_ = Dense(10, activation = "softmax")(mod_)
    
    #eturn pred_
    base_model = VGG16(weights = None, include_top = False)
    x = base_model.output
    x = residual_block(x, 16,16, use_shortcut_ = True)
    x = GlobalAveragePooling2D()(x)
    pred = Dense(10, activation = "softmax")(x)
    model_imagenet = Model(input=base_model.input, output = pred)
    return model_imagenet

In [9]:
model_imagenet=model_build()
model_imagenet.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model_imagenet.summary()

c:\users\chandrachurh\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, None, None, 6 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, None, None, 6 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [11]:
from keras.utils import plot_model
plot_model(model_imagenet, to_file='model_2.png')

In [12]:
nepoch = 50
batch_size = 16

#Applying early stopping
earlyStopping=EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')

lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=2, verbose=1)

save_path="weights_2.h5"
checkpoint = ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

c:\users\chandrachurh\appdata\local\programs\python\python35\lib\site-packages\keras\callbacks.py:1016: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [13]:
model_imagenet.fit(X_train, y_train, epochs=nepoch, batch_size = batch_size, validation_split = 0.1, shuffle=1, callbacks = [lr_reduce, checkpoint], verbose = 2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
 - 324s - loss: 0.2840 - acc: 0.9005 - val_loss: 0.2975 - val_acc: 0.9003

Epoch 00001: val_acc improved from -inf to 0.90026, saving model to weights_2.h5
Epoch 2/50
 - 327s - loss: 0.2544 - acc: 0.9038 - val_loss: 0.2454 - val_acc: 0.9054

Epoch 00002: val_acc improved from 0.90026 to 0.90540, saving model to weights_2.h5
Epoch 3/50
 - 334s - loss: 0.2346 - acc: 0.9092 - val_loss: 0.2420 - val_acc: 0.9085

Epoch 00003: val_acc improved from 0.90540 to 0.90850, saving model to weights_2.h5
Epoch 4/50
 - 325s - loss: 0.2187 - acc: 0.9153 - val_loss: 0.2119 - val_acc: 0.9182

Epoch 00004: val_acc improved from 0.90850 to 0.91822, saving model to weights_2.h5
Epoch 5/50
 - 329s - loss: 0.2040 - acc: 0.9218 - val_loss: 0.1874 - val_acc: 0.9293

Epoch 00005: val_acc improved from 0.91822 to 0.92928, saving model to weights_2.h5
Epoch 6/50
 - 322s - loss: 0.1876 - acc: 0.9293 - val_loss: 0.1705 - val_acc: 0.9333

Epoch 00006: val_a

In [12]:
model_imagenet.load_weights("weights_2.h5")

In [13]:
model_imagenet.evaluate(X_train,y_train, verbose =1)

50000/50000 [==============================] - ETA: 45 - ETA: 46 - ETA: 45 - ETA: 44 - ETA: 44 - ETA: 43 - ETA: 43 - ETA: 43 - ETA: 43 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA

[0.045669403398633006, 0.9842600026321411]